In [ ]:
# set package parent path
import os
import sys

def get_parent_path(parent_folder_name: str = "src"):
    cur_path = os.path.abspath('')
    parent_path = cur_path[:cur_path.find(parent_folder_name) + len(parent_folder_name)]

    return parent_path   

parent_path = get_parent_path()
sys.path.append(parent_path)

In [ ]:
import os

from huggingface_hub import login
from transformers import (AutoTokenizer, DataCollatorForTokenClassification,
                          EarlyStoppingCallback, IntervalStrategy, Trainer,
                          TrainingArguments)

from data.preprocessing.base_model_dataset.bilstm_dataset import \
    BiLSTMBioDataset
from data.preprocessing.task_dataset.srl_dataset import SrlBioDataset
from data.preprocessing.task_dataset.task_dataset import DatasetSize
from models.base.biLSTM import BiLSTM
from models.config import BiLSTMConfig
from utils.metric.srl import compute_metrics
from utils.trainer import BaseModelArgs, setup_before_train

# Setup

In [ ]:
srl_dataset = SrlBioDataset(
    model_name=args.encoder_model_name,
    label_file_name=f"{args.parent_path}/data/processed/{args.dataset_size}_bio_srl_label.csv",
)
srl_dataset = srl_dataset.from_cache(f"{args.parent_path}/data/processed/{args.dataset_size}_bio_srl.csv")

bio_dataset = BiLSTMBioDataset(task_dataset=srl_dataset)

bio_datadict_train = bio_dataset.get_train_datasets()
bio_datadict_val = bio_dataset.get_valid_datasets()

srl_label_vectorizer = srl_dataset.get_label_vectorizer(f"{args.parent_path}/data/processed/{args.dataset_size}_bio_srl_label.csv")

tokenizer = AutoTokenizer.from_pretrained(args.encoder_model_name)

config = BiLSTMConfig(
    vocab_size=tokenizer.vocab_size,
    num_labels=len(srl_label_vectorizer.label_vocab), 
    id2label=srl_label_vectorizer.label_vocab.rev_vocab, 
    label2id=srl_label_vectorizer.label_vocab.vocab,
    hidden_dim=768,
)

model = BiLSTM(config)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

def compute_metrics_callback(p):
    return compute_metrics(p, srl_label_vectorizer.label_vocab)

training_args = TrainingArguments(
    output_dir=args.model_save_dir,
    logging_dir=args.logging_dir,
    learning_rate=args.lr,
    per_device_train_batch_size=args.batch_size,
    per_device_eval_batch_size=args.batch_size,
    num_train_epochs=args.epochs,
    logging_steps=args.eval_steps,
    eval_steps=args.eval_steps,
    save_steps=args.eval_steps,
    max_steps=4500,
    evaluation_strategy=IntervalStrategy.STEPS,
    metric_for_best_model="eval_f1",
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_private_repo=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=bio_datadict_train,
    eval_dataset=bio_datadict_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_callback,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
trainer.train()